Walk trees part 2: walks and selections
----------
- Author: [Timothy Hobbs](https://timothy.hobbs.cz)
- License: [CC-BY-NC 4.0](https://creativecommons.org/licenses/by-nc/4.0/)
- Written: 1.2020
- Send PRs to: [github](https://github.com/timthelion/rust-jupyter)

[project homepage](https://github.com/timthelion/petgraph-fsm) [ipynb](./walk-trees-part2-walks.ipynb)

Walk trees are a novel algorithm for selecting regions of graphs.

A walk is a connected subgraph of a graph. It is selected by selecting an alternating sequence of vertex's and edges. 


In [2]:
:dep petgraph-examples = {path="./petgraph-examples"}

In [3]:
:dep petgraph-evcxr = {path="./petgraph-evcxr"}

In [4]:
:dep petgraph-fsm = {path="./petgraph-fsm"}

In [5]:
:dep petgraph = {path="./petgraph"}

In [6]:
use std::collections::HashMap;
use petgraph::*;
use petgraph::data::*;
use petgraph::visit::*;
use petgraph::graph::*;
use std::hash::Hash;

In [14]:
pub struct Overlay<'a, G, E, N, EW, NW, ER, A>
where
    G: GraphBase<EdgeId = E, NodeId = N> + Data<EdgeWeight = EW, NodeWeight = NW>,
    ER: EdgeRef<Weight = EW, EdgeId = E, NodeId = N>,
    E: Copy + Eq + Hash,
    N: Copy + Eq + Hash,
{
    nodes: HashMap<N, A>,
    edges: HashMap<E, A>,
    edge_refs: HashMap<E, ER>,
    graph: &'a G,
}

In [20]:
pub type Selection<'a, G, E, N, EW, NW, ER> = Overlay<'a, G, E, N, EW, NW, ER, ()>;

In [46]:
enum Phase<'a, N, E> {
    Nodes(std::collections::hash_map::Iter<'a, N, ()>),
    Edges(std::collections::hash_map::Iter<'a, E, ()>),
}

pub struct SelectedItems<'a, G, E, N, EW, NW, ER>
where
    G: GraphBase<EdgeId = E, NodeId = N> + Data<EdgeWeight = EW, NodeWeight = NW> + DataMap,
    ER: EdgeRef<Weight = EW, EdgeId = E, NodeId = N>,
    E: Copy + Eq + Hash,
    N: Copy + Eq + Hash,
{
    selection: &'a Selection<'a, G, E, N, EW, NW, ER>,
    phase: Phase<'a, N, E>,
    node_indexes: HashMap<N, usize>,
}

In [51]:
impl<'a, G, E, N, EW, NW, ER> Iterator for SelectedItems<'a, G, E, N, EW, NW, ER>
where
    G: GraphBase<EdgeId = E, NodeId = N> + Data<EdgeWeight = EW, NodeWeight = NW> + DataMap,
    ER: EdgeRef<Weight = EW, EdgeId = E, NodeId = N>,
    G::NodeWeight: Clone,
    G::EdgeWeight: Clone,
    E: Copy + Eq + Hash,
    N: Copy + Eq + Hash,
{
    type Item = Element<G::NodeWeight, G::EdgeWeight>;
    
    fn next(&mut self) -> Option<Self::Item> {
        if let Phase::Nodes(ref mut nodes) = self.phase {
            match nodes.next() {
                Some(node) =>  {
                    self.node_indexes.insert(*node.0, self.node_indexes.len());
                    return Some(Element::Node {
                        weight: self.selection.graph.node_weight(*node.0).unwrap().clone()
                    })
                },
                None => self.phase = Phase::Edges(self.selection.edges.iter()),
            }
        }
        if let Phase::Edges(ref mut edges) = self.phase {
            match edges.next() {
                Some(edge) => return match self.selection.edge_refs.get(edge.0) {
                    Some(edge_ref) => Some(Element::Edge {
                        weight: edge_ref.weight().clone(),
                        source: *self.node_indexes.get(&edge_ref.source()).unwrap(),
                        target: *self.node_indexes.get(&edge_ref.target()).unwrap(),
                    }),
                    None => return None,
                },
                None => return None,
            }
        }
        None
    }
}

In [11]:
impl<'a, G, E, N> Selection<'a, G, E, N>
where
    G: GraphBase<EdgeId = E, NodeId = N>,
    E: Copy + Eq + Hash,
    N: Copy + Eq + Hash,
{
    /*
    pub fn draw_selection() {
        
    }
    */
    pub fn select_edge<'b>(&'b mut self, edge: EdgeReference) {
        self.edges.insert(edge, ());        
    }
    pub fn select_node<'b>(&'b mut self, node: N) {
        self.nodes.insert(node, ());
    }
    pub fn deselect_edge<'b>(&'b mut self, edge: E) {
        self.edges.remove(&edge);
    }
    fn deselect_node<'b>(&'b mut self, node: N) {
        self.nodes.remove(&node);
    }
}

Error: variable `cellA1` should have a snake case name

Error: variable `cellA2` should have a snake case name

Error: variable `cellA3` should have a snake case name

Error: variable `cellB1` should have a snake case name

Error: variable `cellB2` should have a snake case name

Error: variable `cellB3` should have a snake case name

Error: variable `cellC1` should have a snake case name

Error: cannot find type `Selection` in this scope

Error: wrong number of type arguments: expected at least 1, found 0